In [109]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import warnings as war

In [110]:
df = pd.read_csv("/Users/satwikbiswas/Documents/Python/Data/winequality-red.csv",sep=";")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [ ]:
from scipy import stats
z = np.abs(stats.zscore(df))
print(z)

In [ ]:
### Z-score based outlier removal
threshold = 3
print(np.where(z > 3))  ## this basically provide the row and col of the positions where z>3 like arrar([row]),arr([col])

In [114]:
df_or = df[(z<3).all(axis=1)] ## .all(axis=1) convert all the true/false to a single one based on row 
print(df_or.shape)
df_or.reset_index(drop=True, inplace=True)

(1451, 12)


In [115]:
# print(df_or.shape)
# df_or.drop_duplicates(inplace=True)
# print(df_or.shape)



In [116]:
X = df_or.drop(["quality"],axis=1)
y = df_or["quality"]
X.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


In [117]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=14,shuffle=True)

print(X_train.shape)
print(X_test.shape)

(1160, 11)
(291, 11)


In [ ]:
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import accuracy_score,r2_score

clf = RandomForestClassifier(n_estimators=200)
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

print("accuracy_score = ",accuracy_score(y_true=y_test,y_pred=y_pred))


accuracy_score =  0.6804123711340206


In [144]:
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
rf_params = {
    "n_estimators": np.linspace(200, 1000, 100).astype(int),   # More options for tree count
    "max_depth": np.linspace(2, 14, 2).astype(int),        # Deeper trees for complex patterns
    "max_features": ["sqrt", "log2", None],         # Feature selection per split
    "bootstrap": [True, False],                     # Whether to use bootstrap sampling
    "min_samples_split": [2, 5, 10],                # Controls when to split a node
    "min_samples_leaf": [1, 2, 4]                   # Minimum samples in a leaf
}

clf = RandomizedSearchCV(estimator=RandomForestRegressor(),param_distributions=rf_params,n_jobs=-1,scoring="r2")
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

print("accuracy_score = ",r2_score(y_true=y_test,y_pred=y_pred))


accuracy_score =  0.5148441471637604


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier


In [ ]:


ada_params = {
    "n_estimators": [50, 100, 200, 300],
    "learning_rate": [0.01, 0.1, 0.5, 1.0],
    "estimator": [DecisionTreeClassifier(max_depth=2), DecisionTreeClassifier(max_depth=3)]
}
gb_params = {
    "n_estimators": [100, 200, 300, 500],              # Number of boosting stages (trees)
    "learning_rate": [0.01, 0.05, 0.1, 0.2],            # Step size shrinkage
    "max_depth": [3, 5, 10, 20],                        # Depth of individual trees
    "min_samples_split": [2, 5, 10],                    # Minimum samples to split a node
    "min_samples_leaf": [1, 2, 4],                      # Minimum samples at a leaf node
    "max_features": ["sqrt", "log2", None],             # Features to consider at each split
    "subsample": [0.6, 0.8, 1.0]                        # Fraction of samples used per tree
}
elasticnet_params = {
    "alpha": [0.001, 0.01, 0.1, 1.0, 10.0],              # Regularization strength (λ)
    "l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9, 1.0],          # Mix between L1 (Lasso) and L2 (Ridge)
    "max_iter": [1000, 5000, 10000],                     # Max number of iterations
    "tol": [1e-4, 1e-3, 1e-2],                           # Tolerance for optimization convergence
    "selection": ["cyclic", "random"],                  # Coordinate descent selection strategy
    "fit_intercept": [True, False]                      # Whether to fit the intercept
}



In [121]:
models=[
    {
        "name":"RandomForest",
        "model":RandomForestClassifier(),
        "param":rf_params
    },
    {
        "name":"AdaBoost",
        "model":AdaBoostClassifier(),
        "param":ada_params
    },
    {
        "name":"GradientBoost",
        "model":GradientBoostingClassifier(),
        "param":gb_params
    },
    # {
    #     "name":"Elastic-net",
    #     "model": ElasticNet(),
    #     "param":elasticnet_params
    # }
]

In [129]:
for model in models:
    trainedModel = RandomizedSearchCV(model["model"],param_distributions=model["param"],cv=5,random_state=12,n_jobs=-1,scoring="accuracy")
    trainedModel.fit(X_train,y_train)
    print("for model "+model["name"])
    print()
    print("the params chosen are: ",trainedModel.best_params_)
    print()
    print("the best score is: ",trainedModel.best_score_)
    print('*'*100)

for model RandomForest

the params chosen are:  {'n_estimators': 500, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 50, 'bootstrap': False}

the best score is:  0.6844827586206896
****************************************************************************************************
for model AdaBoost

the params chosen are:  {'n_estimators': 50, 'learning_rate': 0.5, 'estimator': DecisionTreeClassifier(max_depth=3)}

the best score is:  0.6129310344827588
****************************************************************************************************


KeyboardInterrupt: 

# Used Car Dataset

In [123]:
df = pd.read_csv("/Users/satwikbiswas/Documents/Python/Data/used_cars.csv")
df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"


In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


In [125]:
df = df.dropna()

In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3269 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         3269 non-null   object
 1   model         3269 non-null   object
 2   model_year    3269 non-null   int64 
 3   milage        3269 non-null   object
 4   fuel_type     3269 non-null   object
 5   engine        3269 non-null   object
 6   transmission  3269 non-null   object
 7   ext_col       3269 non-null   object
 8   int_col       3269 non-null   object
 9   accident      3269 non-null   object
 10  clean_title   3269 non-null   object
 11  price         3269 non-null   object
dtypes: int64(1), object(11)
memory usage: 332.0+ KB


In [127]:
df["model"].value_counts()

model
M3 Base                 29
F-150 XLT               22
Corvette Base           22
Camaro 2SS              17
Mustang GT Premium      16
                        ..
XTS Premium              1
Terrain Denali           1
Mazda3 i Sport           1
Equinox LT               1
Continental GT Speed     1
Name: count, Length: 1614, dtype: int64